# Air Stability

The [GNoME](https://www.nature.com/articles/s41586-023-06735-9) dataset measures zero-temperature stability of the crystal structures. In this colab, we showcase how to extend this to compute air stability via Pymatgen's tooling for GrandPotentialPhaseDiagram and InterfacialReactivity classes.

Note, this colab showcases how to compute the decomposition energies or reaction en

# Import Libraries

In [ ]:
!pip install pymatgen

In [ ]:
from typing import List, Tuple

import itertools
import json
import pandas as pd

import pymatgen as mg
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.analysis import phase_diagram

## Download the Dataset

In [ ]:
PUBLIC_LINK = "https://storage.googleapis.com/"
BUCKET_NAME = "gdm_materials_discovery"

FOLDER_NAME = "gnome_data"
FILES = (
    "stable_materials_summary.csv",
)

EXTERNAL_FOLDER_NAME = "external_data"
EXTERNAL_FILES = (
    "external_materials_summary.csv"
)

def download_from_link(link: str, output_dir: str):
  """Download a file from a public link using wget."""
  os.system(f"wget {link} -P {output_dir}")

parent_directory = os.path.join(PUBLIC_LINK, BUCKET_NAME)
for filename in FILES:
  public_link = os.path.join(parent_directory, FOLDER_NAME, filename)
  download_from_link(public_link, '.')

for filename in EXTERNAL_FILES:
  public_link = os.path.join(parent_directory, EXTERNAL_FOLDER_NAME, filename)
  download_from_link(public_link, '.')

## Preprocess the GNoME Dataset



In [ ]:
gnome_crystals = pd.read_csv('stable_materials_summary.csv', index_col=0)
gnome_crystals

,Unnamed: 0,Composition,MaterialId,Reduced Formula,Elements,NSites,Volume,Density,Point Group,Space Group,...,Corrected Energy,Formation Energy Per Atom,Decomposition Energy Per Atom,Dimensionality Cheon,Bandgap,Is Train,Decomposition Energy Per Atom All,Decomposition Energy Per Atom Relative,Decomposition Energy Per Atom MP,Decomposition Energy Per Atom MP OQMD
0,234772,Ac10Ag8Au12,719c008190,Ac5(Ag2Au3)2,"['Ag', 'Au', 'Ac']",30,811.0171,11.2541,m,Cm,...,-122.5326,-0.6458,-0.1537,3D,NaN,True,0.0000,-0.0038,-0.1537,-0.0826
1,312926,Ac10Al4Os6,975d473348,Ac5Al2Os3,"['Al', 'Os', 'Ac']",20,549.9835,10.6257,23,I2_13,...,-127.1471,-0.1794,-0.0108,3D,0.0040,True,0.0000,-0.0011,-0.0107,-0.0107
2,189330,Ac10As1Sn6,5b98cce302,Ac10Sn6As,"['As', 'Sn', 'Ac']",17,638.2733,7.9536,-3m,P-31m,...,-82.5133,-0.7403,-0.4838,3D,NaN,True,0.0000,-0.0003,-0.4838,-0.2422
3,390242,Ac10As9Te1,bcad131283,Ac10TeAs9,"['As', 'Te', 'Ac']",20,672.3420,7.5869,-1,P-1,...,-116.3520,-1.5032,-1.4232,3D,NaN,True,0.0000,-0.0086,-1.4231,-0.0086
4,356097,Ac10Au1Os3,ac2f203221,Ac10Os3Au,"['Os', 'Au', 'Ac']",14,487.9847,10.3367,1,P1,...,-80.3566,-0.1563,-0.0907,3D,NaN,True,0.0000,-0.0023,-0.0907,-0.0464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529315,178951,V3W5,56959be1e1,V3W5,"['V', 'W']",8,120.7753,14.7393,mmm,Cmmm,...,-92.7376,-0.0869,-0.0094,3D,0.0046,True,-0.0000,-0.0029,-0.0094,-0.0029
529316,300749,V8W3,9179586bc7,V8W3,"['V', 'W']",11,156.3502,10.1857,2/m,C2/m,...,-112.0538,-0.0462,-0.0088,3D,NaN,True,0.0004,0.0004,-0.0088,0.0004
529317,501620,Y1Zn4Zr1,f254757fb4,YZrZn4,"['Zn', 'Y', 'Zr']",6,114.3604,6.4145,6/mmm,P6/mmm,...,-22.2242,-0.3618,-0.0151,3D,0.0040,False,0.0000,-0.0043,-0.0151,-0.0043
529318,319699,Y6Zn44,9aa63becc5,Y3Zn22,"['Zn', 'Y']",50,823.5730,6.8783,4/mmm,I4_1/amd,...,-107.3622,-0.2627,0.0001,NaN,NaN,NaN,0.0001,0.0001,0.0001,0.0001


In [ ]:
reference_crystals = pd.read_csv('external_materials_summary.csv')
reference_crystals

,Unnamed: 0,Composition,MaterialId,Reduced Formula,Elements,NSites,Corrected Energy,Decomposition Energy Per Atom All,Decomposition Energy Per Atom MP
0,29940,Ac4,98dee9c38d,Ac,['Ac'],4.0,-16.484703,0.0,-7.584912e-07
1,23204,Ac1Ag2Ge2,132e8b7d0b,Ac(AgGe)2,"['Ac', 'Ag', 'Ge']",5.0,-21.057337,0.0,-1.165034e-01
2,44185,Ac1Ag2Sn2,c0cf7ad279,Ac(AgSn)2,"['Ac', 'Ag', 'Sn']",5.0,-19.976854,0.0,-1.451672e-01
3,6084,Ac1Ag8Al4,5338e797c2,Ac(AlAg2)4,"['Ac', 'Al', 'Ag']",13.0,-44.512306,0.0,-6.966974e-02
4,3564,Ac2Al4Au4,c6efd90a00,Ac(AlAu)2,"['Ac', 'Al', 'Au']",10.0,-43.610382,0.0,-2.132195e-01
...,...,...,...,...,...,...,...,...,...
54472,47097,Pd2Zn2Zr2,b19a5a64ff,ZrZnPd,"['Zr', 'Zn', 'Pd']",6.0,-33.405891,0.0,-3.882178e-02
54473,10048,Pd2Zn1Zr1,dc694cf185,ZrZnPd2,"['Zr', 'Zn', 'Pd']",4.0,-22.807571,0.0,-1.617029e-02
54474,56135,Pt2Zn2Zr2,5b263880de,ZrZnPt,"['Zr', 'Zn', 'Pt']",6.0,-36.715942,0.0,-7.164972e-02
54475,2488,Rh3Zn3Zr3,5f6876c555,ZrZnRh,"['Zr', 'Zn', 'Rh']",9.0,-57.113941,0.0,-2.625296e-03


In [ ]:
def annotate_chemical_system(crystals: pd.DataFrame) -> pd.DataFrame:
  chemical_systems = []
  for i, e in enumerate(crystals['Elements']):
    try:
      # replace single quotes with double quotes to avoid having to use python eval
      chemsys = json.loads(e.replace("'", '"'))
      chemical_systems.append(tuple(sorted(chemsys)))
    except:
      print(e)
  crystals['Chemical System'] = chemical_systems
  return crystals

In [ ]:
# Preprocess crystal structure
gnome_crystals = annotate_chemical_system(gnome_crystals)
reference_crystals = annotate_chemical_system(reference_crystals)

In [ ]:
all_crystals = pd.concat([gnome_crystals, reference_crystals], ignore_index=True)
required_columns = ['Composition', 'NSites', 'Corrected Energy', 'Formation Energy Per Atom', 'Chemical System']
minimal_entries = all_crystals[required_columns]
grouped_entries = minimal_entries.groupby('Chemical System')

## Choose a Random Structure

A random structure is chosen to compute the decomposition energy for.

In [ ]:
# Choose a sample crystal
binaries = gnome_crystals[gnome_crystals['Chemical System'].map(len) == 2]
sample = binaries.sample()

In [ ]:
# The sample we have chosen and would like to visualize
sample

,Unnamed: 0,Composition,MaterialId,Reduced Formula,Elements,NSites,Volume,Density,Point Group,Space Group,...,Formation Energy Per Atom,Decomposition Energy Per Atom,Dimensionality Cheon,Bandgap,Is Train,Decomposition Energy Per Atom All,Decomposition Energy Per Atom Relative,Decomposition Energy Per Atom MP,Decomposition Energy Per Atom MP OQMD,Chemical System
449022,293472,I17Tc3,8df1ed396f,Tc3I17,"['Tc', 'I']",20,974.5844,4.1768,1,P1,...,-0.3696,-0.148,1D,0.2022,True,0.0,-0.0761,-0.148,-0.0761,"(I, Tc)"


In [ ]:
chemsys = sample['Chemical System'].item()

## Stability with Respect to Oxygen via GrandPotentialPhaseDiagram

The code below provides an example of computing the stability with respect to the oxygen on the GrandPotentialPhaseDiagram.

In [ ]:
from pymatgen.analysis import interface_reactions

In [ ]:
element = 'O'
temperature = 300
pressure = 21200
chempot_oxygen = interface_reactions.InterfacialReactivity.get_chempot_correction(
    element, temperature, pressure)
u_o = -4.95 + chempot_oxygen

In [ ]:
oxygen_chemsys = chemsys + ('O',)
chempots = {mg.core.Element('O'): u_o}

In [ ]:
def collect_phase_diagram_entries(
    chemsys: Tuple[str, ...],
    grouped_entries: pd.core.groupby.generic.DataFrameGroupBy,
    minimal_entries: pd.DataFrame
) -> List[ComputedEntry]:
  phase_diagram_entries = []
  for length in range(len(chemsys) + 1):
    for subsystem in itertools.combinations(chemsys, length):
      subsystem_key = tuple(sorted(subsystem))
      subsystem_entries = grouped_entries.groups.get(subsystem_key, [])
      if len(subsystem_entries):
        phase_diagram_entries.append(minimal_entries.iloc[subsystem_entries])
  phase_diagram_entries = pd.concat(phase_diagram_entries)

  mg_entries = []

  for _, row in phase_diagram_entries.iterrows():
    composition = row['Composition']
    formation_energy = row['Corrected Energy']
    entry = ComputedEntry(composition, formation_energy)
    mg_entries.append(entry)

  return mg_entries

In [ ]:
gnome_grand_diagram_entries = collect_phase_diagram_entries(oxygen_chemsys, grouped_entries, all_crystals)

In [ ]:
gnome_grand_diagram = phase_diagram.GrandPotentialPhaseDiagram(
    gnome_grand_diagram_entries,
    chempots=chempots)

In [ ]:
sample_entry = ComputedEntry(
    sample['Composition'].item(),
    sample['Corrected Energy'].item(),
)

sample_grand_entry = phase_diagram.GrandPotPDEntry(
    sample_entry,
    chempots=chempots
)

In [ ]:
decomposition, decomposition_energy = gnome_grand_diagram.get_decomp_and_e_above_hull(sample_grand_entry)

In [ ]:
# The decomposition energy of the provided structure with respect to oxgyen
print(f'Decomposition energy with oxgyen: {decomposition_energy}')

Decomposition energy with oxgyen: 1.5718342511757761


## Stability with Respect to CO2 and H2O via InterfacialReactivity

We also use the interfacial reactivity diagram in order to check stability with respect to carbon dioxide and water.

In [ ]:
gnome_phase_diagram_entries = collect_phase_diagram_entries(
    chemsys + ('H', 'C', 'O'),
    grouped_entries,
    all_crystals
)
gnome_phase_diagram = phase_diagram.PhaseDiagram(gnome_phase_diagram_entries)

In [ ]:
carbon_dioxide = mg.core.Composition('CO2')
gnome_co2_rxns = interface_reactions.InterfacialReactivity(
    sample_entry.composition,
    carbon_dioxide,
    gnome_phase_diagram,
    use_hull_energy=True,
)
co2_stability = gnome_co2_rxns.minimum[1]
co2_stability

0.0

In [ ]:
water = mg.core.Composition('H2O')
gnome_h2o_rxns = interface_reactions.InterfacialReactivity(
    sample_entry.composition,
    water,
    gnome_phase_diagram,
    use_hull_energy=True,
)
h2o_stability = gnome_h2o_rxns.minimum[1]
h2o_stability

0.0